In [ ]:
#!cd ../../ && python3 setup.py build_ext --inplace
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from random import random
from numpy.random import randint
from scipy.ndimage import gaussian_filter
from nanopyx.core.transform.binning import rebin2d
from nanopyx.core.transform.image_add_random_noise import (
    addMixedGaussianPoissonNoise,
    addPerlinNoise,
    addRamp,
)

# generate some random ground truth
w = 640
h = 640
image_gt = np.ones((w, h), dtype="float32") * 1000
addPerlinNoise(image_gt, amp=1000, f=100, octaves=3)

image_ref = gaussian_filter(image_gt, 15)
image_ref = rebin2d(image_ref, 10, mode="mean")
# addRamp(image_ref, 1000)
addMixedGaussianPoissonNoise(image_ref, 10, 10)

image_sr = image_gt.copy()
image_sr = gaussian_filter(image_sr, 3)
# image_sr = rebin2d(image_sr, 10, mode="mean")


import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure with 3 subplots arranged in a row
fig, axs = plt.subplots(1, 3, figsize=(24, 24))
sns.heatmap(image_gt, ax=axs[0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(image_ref, ax=axs[1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(image_sr, ax=axs[2], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[0].set_title("GT")
axs[1].set_title("Ref")
axs[2].set_title("SR")

# Show the figure
plt.show()


In [ ]:
from nanopyx.core.sr.error_map import ErrorMap

squirrelErrorMap = ErrorMap()
squirrelErrorMap.optimise(image_ref, image_sr)

print(f"RSE: {squirrelErrorMap.getRSE()}")
print(f"RSP: {squirrelErrorMap.getRSP()}")
print(f"Sigma: {squirrelErrorMap.getSigma()}")

# Create a figure with 3 subplots arranged in a row
fig, axs = plt.subplots(1, 3, figsize=(24, 24))

sns.heatmap(squirrelErrorMap.imRSE, ax=axs[0], cmap='viridis', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(image_ref, ax=axs[1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(squirrelErrorMap.imSRIntensityScaledBlurred, ax=axs[2], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[0].set_title("Error Map")
axs[1].set_title("SR blurred")
axs[2].set_title("Ref")

# Show the figure
plt.show()
